# GitHub Mining

### Einführung

Gleich zu Beginn soll die Frage geklärt werden, wofür GitHub verwendet wird und welchen Nutzen man aus dem Sammeln der Daten ziehen kann.
>GitHub ist ein webbasierter Online-Dienst, der Software-Entwicklungsprojekte auf seinen Servern bereitstellt (Filehosting). Im Gegensatz zu anderen Dienstleistern zur Verwaltung quelloffener Software steht auf GitHub nicht das Projekt als Sammlung von Quellcode im Zentrum, sondern der Nutzer mit seinen Quelltext-Datenbanken, den sogenannten repositories. Auch das Erstellen (branchen) und Zusammenführen (mergen) von Abspaltungen (forks) steht besonders im Mittelpunkt. Die sogenannten forks machen das Mitentwickeln bei fremden Projekten besonders einfach: Um dort einen Beitrag beizusteuern, wird das Repository zunächst abgespalten, dann werden die zu übernehmenden Änderungen hinzugefügt und dem Besitzer des Originals eine Anfrage (pull request) gestellt, die Änderungen zu übernehmen. Damit wird unter Berücksichtigung der Besonderheiten verteilter Versionskontrollsysteme ein soziales Netzwerk geschaffen, was sich auch in den aus „normalen“ sozialen Netzwerken bekannten Funktionen „Beobachten“ oder „Folgen“ zeigt.


### Überblick

Der Schwerpunkt dieses Kapitels wird sein, wie man die Beziehungen zwischen GitHub, den users und repositories als interest graph behandelt.
* GitHubs Entwicklerplattform und wie man API requests erstellt
* Graphen-Schemas und Modellierung von property graphs mit NetworkX
* Konzept des interest graphs und wie ein interest graph anhand GitHub-Daten erstellt wird
* Abfragen von property graphs mittels NetworkX

## Begriffserklärungen

### GitHub API
>Da die [GitHub developer site](https://developer.github.com/v3/) eine viel zu umfassende Dokumentation aller APIs bietet, werden wir uns mit ein paar wenigen APIs auseinandersetzen, die wir benötigen um Daten für interest graphs zu sammeln. Die wichtigsten Stammfunktionen für GitHub sind *users* und *projects*. Ein GitHub-User hat ein öffentliches Profil, welches generell ein oder mehrere repositories beinhält, die entweder erstellt oder *geforked* wurden. Zusätzlich können user andere Projekte bookmarken oder einen Stern vergeben, wodurch sie zu sogenannten *stargazers* dieses Projekts werden.

### repository
>Ein Repository (oder kurz repo) kann bei GitHub einfach als „Projekt“ verstanden werden. Sämtliche Dateien für ein Software-Projekt werden in einem Repository abgelegt.

### stargazer
>*Starring* oder *stargazen* eines repositories erlaubt es, beim jeweiligen Projekt auf dem Laufenden zu bleiben, selbst wenn man nicht daran beteiligt ist. Die Sterne werden neben dem repository angezeigt und sollen veranschaulichen, wie groß das Interesse anderer user an diesem repository ist.

### fork
>Da alle öffentlichen Git-Projekte unter OpenSource-Lizenz stehen, kann auch jeder einen eigenen Ableger eines repositories, einen sogenannten fork starten. Dort kann jeder privat vor sich hin entwickeln und seine Version am Ende wieder dem ursprünglichen Projekt zuführen (pull request stellen) – oder aber einfach eine eigenständige Variante verbreiten.

## 1. Erstellen der GitHub API-Verbindung

Es gibt die Möglichkeit bestätigte (authenticated) und unbestätigte (unauthenticated) requests auszugeben:
* auth:   5.000 requests per hour (großzügig!)
* unauth: 60 requests per hour (ausreichend zum herumexperimentieren)

Für unser GitHub Mining Script arbeiten wir mit dem Profil von Matthew A.Russell, der mit seinen Publikationen 
* Mining the social Web &
* Mining the social Web 2nd Edition

diesbezüglich einen großen Bekanntheitsgrad auf GitHub erlangt hat. Die große Zahl an *stargazern* und *followern* eignet sich gut für statistische Auswertungen.
>**user:** ptwobrussell

>**repository:** Mining-the-Social-Web-2nd-Edition

Wir verwenden die Bibliothek *requests*, die es erlaubt HTTP-Anfragen mittels Python zu versenden. Zusätzlich können *headers* und diverse Parameter mit ausgegeben werden.

In [1]:
import requests
import json

**Bsp.: 1** - Erstellen des persönlichen API access Tokens zum bestätigten Verbindungsaufbau

In [2]:
from getpass import getpass

In [5]:
username = 'xxx' # Dein GitHub username
password = 'xxx' # Dein GitHub password
url = 'https://api.github.com/authorizations' 
note = 'xxx' # Tokenbezeichnung
post_data = {'scopes':['repo'],'note': note }

response = requests.post(url, auth = (username, password), data = json.dumps(post_data))   

print("API response:", response.text) # Hier ist der Token bereits auszulesen
print()
print("Dein OAuth Token lautet:", response.json()['token']) # Zur besseren Sichtbarkeit

API response: {"id":111840130,"url":"https://api.github.com/authorizations/111840130","app":{"name":"test","url":"https://developer.github.com/v3/oauth_authorizations/","client_id":"00000000000000000000"},"token":"b647f0380da1f75c9a5dec8363922632a4b147ad","hashed_token":"b50682118dd5d19552a55184a3e2b55f26f1890d465d96f9eef74abc5ebd2d5a","token_last_eight":"a4b147ad","note":"test","note_url":null,"created_at":"2017-06-29T14:51:32Z","updated_at":"2017-06-29T14:51:32Z","scopes":["repo"],"fingerprint":null}

Dein OAuth Token lautet: b647f0380da1f75c9a5dec8363922632a4b147ad


**Hinweis:**
Dieser Token kann [hier](https://github.com/settings/tokens) wieder entfernt werden. Mithilfe des Tokens ist es für jeden möglich, sich unabhängig von username und password, mit dem jeweiligen GitHub-Profil anzumelden. Sollte der Token versehentlich auf GitHub mit hochgeladen werden, erkennt GitHub das und verwirft diesen automatisch. **Das gilt nicht für die restlichen Nutzerdaten!**

**Bsp.: 2** - Erstellen direkter HTTP requests an die GitHub API

Hier wird ein unbestätigter request ausgeführt, der keinen ?access_token=xxx query string beinhaltet.

In [6]:
url = "https://api.github.com/repos/ptwobrussell/Mining-the-Social-Web-2nd-Edition/stargazers"
response = requests.get(url)

In [7]:
for (k,v) in response.headers.items():
    print(k, "=>", v) # Zeige header

Server => GitHub.com
Date => Thu, 29 Jun 2017 14:52:34 GMT
Content-Type => application/json; charset=utf-8
Transfer-Encoding => chunked
Status => 200 OK
X-RateLimit-Limit => 60
X-RateLimit-Remaining => 57
X-RateLimit-Reset => 1498751366
Cache-Control => public, max-age=60, s-maxage=60
Vary => Accept, Accept-Encoding
ETag => W/"bce16fe7a2c46897ecebc2f163daa4f7"
X-GitHub-Media-Type => github.v3; format=json
Link => <https://api.github.com/repositories/9784365/stargazers?page=2>; rel="next", <https://api.github.com/repositories/9784365/stargazers?page=78>; rel="last"
Access-Control-Expose-Headers => ETag, Link, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval
Access-Control-Allow-Origin => *
Content-Security-Policy => default-src 'none'
Strict-Transport-Security => max-age=31536000; includeSubdomains; preload
X-Content-Type-Options => nosniff
X-Frame-Options => deny
X-XSS-Protection => 1; mode=block
X-Runti

**GitHub gibt uns bereits hier nützliche Informationen zurück, wie...**
* den Status des requests [ok]
* das X-RateLimit-Limit => 60
* und das X-RateLimit-Remaining

## 1.1. Erstellen von GiHub API Requests

Obwohl es möglich ist, Bibliotheken wie *request* zu verwenden und die Informationen selbst herauszuanalysieren, ist es für uns einfacher Bibliotheken wie PyGithub zu verwenden

**Hinweis:** 

Wir werden für den weiteren Gebrauch die Python Bibliothek **PyGitHub** verwenden

> **pip install pygithub**

**Bsp.: 3** - Verwenden von PyGithub, um die Anzahl *stargazer* eines bestimmten repositories abzufragen

In [9]:
from github import Github

In [11]:
ACCESS_TOKEN = 'b647f0380da1f75c9a5dec8363922632a4b147ad' # Hier das erstellte token einfügen

USER = 'ptwobrussell' # Hier den  GitHub user...
REPO = 'Mining-the-Social-Web-2nd-Edition' # ...samt gewünschtem repository einfügen

client = Github(ACCESS_TOKEN, per_page=100)
user = client.get_user(USER)
repo = user.get_repo(REPO)

stargazers = [ s for s in repo.get_stargazers() ]
print("Anzahl der stargazer:", len(stargazers))

Anzahl der stargazer: 2329


Wir haben nun eine authentifizierte Verbindung zu GitHub aufgebaut und erfolgreich die Anzahl der stargazer abgefragt.

Eine genaue Beschreibung der "API implementation details" lässt sich in der [PyGithub Dokumentation](http://pygithub.github.io/PyGithub/v1/index.html) nachlesen.

## 2. Modeling Data with Property Graphs

Ein property graph ist eine Datenstruktur, die *entities* (Objekte) mit *nodes* und die Beziehung zwischen den Objekten mit *edges* darstellt

**Figur 2.1** - zeigt ein sehr einfaches Beispiel eines property graphs mit zwei *nodes*, welche eindeutig durch *X* und *Y* identifizierbar sind. Die Beziehung dazwischen ist nicht näher beschrieben

![alt text](https://github.com/nick90/github-mining/blob/master/pg.png?raw=true "propertygraph")
    Figur 2.1

**Hinweis:**

Als Code ausgedrückt kann ein einfacher property graph wie in **Bsp.: 4** über **networkx** konstruiert werden.

*Die Installation der NetworkX-Bibliothek erfolgt über das Terminal*
>**pip install networkx**

**Bsp.: 4** - Erstellen eines einfachen property graph

In [16]:
import networkx as nx

In [17]:
# Erstellen des gezielten Graphen
g = nx.DiGraph()

# Erstellen einer edge von X nach Y
g.add_edge('X', 'Y')

# Print von statistischen Infos
print(nx.info(g))
print()

# Print der edges und nodes
print("Nodes:", g.nodes())
print("Edges:", g.edges())
print()

# node-properties
print("X props:", g.node['X'])
print("Y props:", g.node['Y'])

# edge properties
print("X=>Y props:", g['X']['Y'])
print()

# Update node property
g.node['X'].update({'prop1' : 'value1'})
print("X props:", g.node['X'])
print()

# Update edge property
g['X']['Y'].update({'label' : 'label1'})
print("X=>Y props:", g['X']['Y'])

Name: 
Type: DiGraph
Number of nodes: 2
Number of edges: 1
Average in degree:   0.5000
Average out degree:   0.5000

Nodes: ['X', 'Y']
Edges: [('X', 'Y')]

X props: {}
Y props: {}
X=>Y props: {}

X props: {'prop1': 'value1'}

X=>Y props: {'label': 'label1'}


**Hinweis:**

Die [NetworkX Dokumentation](http://networkx.github.io/) bietet eine Vielzahl an hilfreichen Beispielen, um den Umgang zu lernen.

**Bsp.: 5** - Erstellen des ego graphs eines repositories und seiner stargazer

In [18]:
g = nx.DiGraph()
g.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)

for sg in stargazers:
    g.add_node(sg.login + '(user)', type='user')
    g.add_edge(sg.login + '(user)', repo.name + '(repo)', type='gazes')

**Bsp.: 6** - Output von NetworkX Informationen

In [23]:
print(nx.info(g))
print()

Name: 
Type: DiGraph
Number of nodes: 2330
Number of edges: 2329
Average in degree:   0.9996
Average out degree:   0.9996



Wir wissen bisher, dass über 2300 User gemeinsames Interesse an social web mining haben, bezogen auf ihr *starring* auf das gemeinsame repository.
Die Zahl an edges ist aufgrund der 1-zu-1-Beziehung zwischen stargazer und repository um 1 kleiner als das der nodes.

**Figur 2.2** - zeigt ein Beispiel eines ego graphs

Verglichen mit den Daten aus **Bsp.: 6** kann man sich hier **einen** großen *node* in der Mitte vorstellen und 2329 *nodes* außen, die über 2329 *edges* eine Verbindung finden

![alt text](https://github.com/nick90/github-mining/blob/master/egograph.png?raw=true "ego graph")
    Figur 2.2

## 3. Erweitern des Graphen mit "Follow" edges für User

Zusätzlich zum *stargazen* und *forken* von repositories, bietet GitHub zudem die Möglichkeit, anderen usern zu *followen*.
Die Chancen sind hoch, dass der User, dessen Repository in großer Zahl von der Community *gestarred* wird, auch selbst beliebt ist. 

Doch wer von den *starring* users ist noch beliebt?

Diese Frage wollen wir mit Abfragen der GitHubs [User Followers API](https://developer.github.com/v3/users/followers/) beantworten.

**Bsp.: 8** - Erweitern des Graphen durch das Hinzufügen von *follows* edges

In [28]:
import sys

In [ ]:
for i, sg in enumerate(stargazers):
    try:
        for follower in sg.get_followers():
            if follower.login + '(user)' in g:
                g.add_edge(follower.login + '(user)', sg.login + '(user)',
                           type='follows')
    except Exception:
        print >> sys.stderr, "Encountered an error fetching followers for", \
            sg.login, "Skipping."
        print >> sys.stderr
    print("Processed", i+1, " stargazers. Num nodes/edges in graph", \
        g.number_of_nodes(), "/", g.number_of_edges())
    print("Rate limit remaining", client.rate_limiting)

**Hinweis:**

Das Data-Mining von über 2300 stargazern dauert einige Zeit. Im folgenden Schritt können die neugewonnenen Daten verwendet werden, um weitere beliebte user herauszufiltern.

**Figur 3.1** - zeigt ein Graphschema, indem GitHub user sowohl an repository als auch andere user interesse zeigen

![alt text](https://github.com/nick90/github-mining/blob/master/pgf.png?raw=true)
Figur 3.1 

**Bsp.: 9** - Updated graph mit follow edges

In [70]:
from operator import itemgetter
from collections import Counter

print(nx.info(g))
print()

# Zahl der benachbarten edges zur node
print(sorted([n for n in g.degree_iter()], key=itemgetter(1), reverse=True)[:10])
print()

#Zahl der beliebten User
c = Counter([e[1] for e in g.edges_iter(data=True) if e[2]['type'] == 'follows'])
popular_users = [ (u, f) for (u, f) in c.most_common() if f > 1 ]
print("Anzahl der beliebten user:", len(popular_users))
print("Top 10 beliebte user:", popular_users[:10])

Name: 
Type: DiGraph
Number of nodes: 55329
Number of edges: 105288
Average in degree:   1.9029
Average out degree:   1.9029

[('Mining-the-Social-Web-2nd-Edition(repo)', 2329), ('angusshire(user)', 636), ('zygmuntz(user)', 570), ('rossant(user)', 536), ('JT5D(user)', 523), ('rohithadassanayake(user)', 523), ('bigsnarfdude(user)', 512), ('swhgoon(user)', 512), ('esafak(user)', 512), ('aburan28(user)', 511)]

Anzahl der beliebten user: 268
Top 10 beliebte user: [('CamDavidsonPilon(user)', 99), ('zygmuntz(user)', 72), ('josephmisiti(user)', 37), ('rossant(user)', 28), ('chdoig(user)', 26), ('clarecorthell(user)', 22), ('vlandham(user)', 19), ('dwillis(user)', 18), ('jadianes(user)', 18), ('hmcuesta(user)', 17)]


## 4. Hinzufügen weiterer repositories zum interest graph

Nun können wir einen weiteren Schritt wagen, und von jedem weiteren user die zusätzlich *starred* repositories zusammensuchen und *edges* erstellen, um gemeinsame Interessen herauszufinden. 

**Bsp.: 10** - Hinzufügen von *starred* repositories zum Graphen

In [ ]:
MAX_REPOS = 500
for i, sg in enumerate(stargazers):
    print(sg.login)
    try:
        for starred in sg.get_starred()[:MAX_REPOS]: # Slice to avoid supernodes
            g.add_node(starred.name + '(repo)', type='repo', lang=starred.language, \
                owner=starred.owner.login)
            g.add_edge(sg.login + '(user)', starred.name + '(repo)', type='gazes')
    except Exception:
        print("Encountered an error fetching starred repos for", sg.login, "Skipping.")

**Hinweis:**

Das Data-Mining von über 2300 stargazern und dessen repositories dauert **noch länger**. 

Im folgenden Schritt können die neugewonnenen Daten verwendet werden, um weitere beliebte repositories herauszufiltern.

**Bsp.: 11** - Updated graph mit repository edges

Hier noch einmal zur Veranschaulichung die Anzahl der neuen *nodes* und *edges*, welche sich vervielfacht haben. Zudem weitere repositories, die aus der Gesamtzahl der *stargazer* gemeinsames Interesse finden.

In [69]:
print(nx.info(g))
print()

repos = [n for n in g.nodes_iter() if g.node[n]['type'] == 'repo']

print(sorted([(n,d)
    for (n,d) in g.in_degree_iter()
        if g.node[n]['type'] == 'repo'], \
            key=itemgetter(1), reverse=True)[:10])

Name: 
Type: DiGraph
Number of nodes: 55329
Number of edges: 105288
Average in degree:   1.9029
Average out degree:   1.9029

[('Mining-the-Social-Web-2nd-Edition(repo)', 2329), ('free-programming-books(repo)', 84), ('d3(repo)', 83), ('Probabilistic-Programming-and-Bayesian-Methods-for-Hackers(repo)', 80), ('tensorflow(repo)', 76), ('ipython(repo)', 75), ('dotfiles(repo)', 73), ('bootstrap(repo)', 71), ('go(repo)', 67), ('awesome-public-datasets(repo)', 58)]


Aus diesen Daten können wir nun noch viele weitere Informationen ziehen, wie folgendes Beispiel zeigt.

**Bsp.: 12** - Programmiersprachen, an denen user *CamDavidsonPilon* Interesse hat

In [60]:
print(list(set([g.node[n]['lang']
    for n in g['CamDavidsonPilon(user)']
        if g['CamDavidsonPilon(user)'][n]['type'] == 'gazes'])))

['Vala', 'Shell', 'Jupyter Notebook', 'Haskell', 'Objective-J', 'Julia', 'C++', None, 'Objective-C', 'CSS', 'C', 'Emacs Lisp', 'CoffeeScript', 'Python', 'JavaScript', 'PHP', 'Ruby', 'HTML', 'VimL', 'Scala', 'Java']


**Bsp.: 12** - Graphenvisualisierung des Sozialen Netzwerks

Mithilfe eines Tools konnte dieser graph aufwändig visualisiert werden.

![alt text](https://github.com/nick90/github-mining/blob/master/fg.png?raw=true "final graph")
Figur 4.1

Wenn alle Daten gesammelt wurden, kann eine interactive Visualisierung der *follow* edges innerhalb der GitHub user für den interest graph angezeigt werden

## Quellen und Literatur
**Matthew A.Russel | O'Reilly**
* Mining the Social Web | Data Mining Facebook, Twitter, Linkedin, Google+, GitHub, and more

**Francesco Pontillo | 31.08.2016**
* https://www.novoda.com/blog/github-data-mining-101/
* https://de.wikipedia.org/wiki/GitHub 
* https://developer.github.com/v3/
* http://pygithub.github.io/PyGithub/v1/index.html
* https://developer.github.com/v3/users/followers/